In [1]:
import os
import sys; sys.path.append("../../../")
from img_pipe.img_pipe import img_pipe
from importlib import reload
reload(img_pipe)
import numpy as np
import scipy.io
%matplotlib inline

# Img_pipe Recon Demo

This tutorial will walk through how to run a recon using the img_pipe package. This includes MRI alignment, CT registration, electrode localization, anatomical labeling, and nonlinear warping. 

### Set-up

In your SUBJECTS_DIR (where you want to place your subjects imaging data), you should make a new directory (with your subject name, i.e. 'EC86'). Then, create subdirectories __*acpc/*__ and __*CT/*__. Place your niftii format T1 MRI file (name it *T1_orig.nii*) in the __*acpc/*__ folder and the CT niftii file (name it *CT.nii*) in __*CT/*__. Your directory structure should now look like this: 

SUBJECTS_DIR/
    * Subj_name/
        * acpc/
            *T1_orig.nii
        * CT/
            * CT.nii

### 1. Alignment of T1 scan to AC-PC axis

*	Alignment of the T1 scan to the anterior commissure-posterior commissure axis is performed in Freeview (Figure 3). Open Freeview and load your unaligned T1 scan in the Volumes tab. To aid in axis alignment, change the cursor style “long” in Preferences  Cursor  style “Long”. You can also change the color if you wish.
*	To adjust the rotation and translation of the image, select Tools  Transform Volume. Adjust the roll (with Y (P-A)) and yaw (with Z (I-S)) as necessary to make sure the head is aligned.  Check the axial view to make sure the eyes show equally in the same slice (see Fig 3A vs. 3B, second panel for unaligned and aligned examples).  Make sure the midsagittal line is vertical in the axial view (see Fig 3A vs. 3B, first and third panels) and in the coronal view.  Choose Sample method “Cubic”.
*	Select the anterior commissure and adjust the pitch of the head so that it is in line with the posterior commissure on the horizontal axis (Fig 3A-B, last panel).
*	Finally, move to the (0, 0, 0) RAS coordinate (not TkReg RAS, just RAS).  In the Transform Volume tool, translate the image until the cursor is at the anterior commissure.   
*	Once you are satisfied that the brain is in a good orientation, click “Save Reg…” and save the transformation matrix in the acpc directory as T1_reorient.lta. Then, click “Save as…” and save the reoriented T1 file as T1.nii in the acpc directory (e.g. /usr/local/freesurfer/subjects/EC1/acpc). 

#### Include image here 

### 2. Create the freeCoG instance for the patient and run preparatory steps

In [44]:
subj_dir = '/home/adam2392/hdd/data/neuroimaging/freesurfer_output/'
subj_dir = "/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/"

fs_dir = '/opt/freesurfer/'
fs_dir = "/Users/adam2392/Documents/freesurfer/"


patient = img_pipe.freeCoG(subj='umf005',
                           hem='rh',
                           subj_dir=subj_dir,
                           fs_dir=fs_dir)
# you can also specify SUBJECTS_DIR and FREESURFER_HOME in your .bash_profile instead of specifying 
# in the *subj_dir* and *fs_dir* arguments.

#### 2a) **`prep_recon()`**
**`prep_recon()`** will set up the directory structure needed before running **`get_recon()`**

In [45]:
patient.prep_recon()

#### 2b) **`get_recon()`** 
**`get_recon()`** will run the Freesurfer command `recon-all`. This will take on the order of hours, depending on your machine's CPU, GPU, and whether you use parallelization.

In [14]:
patient.get_recon()

#### 2c) check pial surfaces against the MRI with **`check_pial()`**
Now, check that the pial surfaces actually correspond to the MRI.

In [13]:
#this will open a freesurfer window with the brain MRI and pial surfaces loaded
patient.check_pial()

#### 2d) create the meshes

Create the triangle-vertex mesh in .mat format with **`convert_fsmesh2mlab()`**. This will save them out to **__Meshes/EC108_lh_pial.mat__** and **__Meshes/EC108_rh_pial.mat__**. To create subcortical meshes, run **`get_subcort()`**, subcortical meshes will be in **__Meshes/subcortical/__**.

In [4]:
patient.convert_fsmesh2mlab()
patient.get_subcort()

Making Meshes Directory
Making lh mesh
Making rh mesh
::: Tesselating freesurfer subcortical segmentations from aseg using aseg2srf... :::
/home/adam2392/Documents/neuroimg_pipeline/img_pipe/img_pipe/SupplementalScripts/aseg2srf.sh
Creating directory /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf004/Meshes/subcortical
::: Converting all ascii segmentations to matlab tri-vert :::


#### 2e) CT Registration

Register the **__CT/CT.nii__** file to **__mri/orig.nii__** file; the registered CT will be saved as **__CT/rCT.nii__**.

Note, that the coregistred CT can just be copy and pasted over from our flirt run. This coregistratino using freesurfer is not that great.

In [27]:
# patient.reg_img('CT.nii','T1.nii')

Computing registration from /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf001/CT/CT.nii to /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf001/mri/T1.nii
Initial guess...
translation : [0. 0. 0.]
rotation    : [0. 0. 0.]

Optimizing using fmin_powell
translation : [1. 1. 1.]
rotation    : [0.01 0.01 0.01]

nmi = 2.0

Optimization terminated successfully.
         Current function value: -2.000000
         Iterations: 1
         Function evaluations: 25
Saving registered CT image as /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf001/CT/rCT.nii


/home/adam2392/.local/lib/python3.6/site-packages/nipy/io/files.py:145: FutureWarning: Default `strict` currently False; this will change to True in a future version of nipy
  ni_img = nipy2nifti(img, data_dtype = io_dtype)


### 3. Electrode localization, anatomical labeling, and nonlinear warping

#### 3a) Marking the electrodes on the CT

Example of identification of electrode coordinates using electrode picker, opened with **`mark_electrodes()`**. 

<img src='files/ElectrodeMarker.png' style='width:700px;'>

**(A)** Demonstrates the process of picking the coordinate for the most posterior inferior grid corner. On the left, the GUI is shown with the electrode selected. The pial surface, rCT, and skull stripped MRI are displayed. The upper left shows the electrode selected in the sagittal view. The upper right shows the coronal view. The bottom right shows an axial view. The lower left displays the intensity projection map of the CT, which is useful for visualizing the entire grid. To save the coordinate, press ‘n’ to name a new device. With the center of the electrode artifact localized by the crosshairs in the axial, sagittal, and coronal views, press ‘e’ to add a point. The coordinates are automatically saved to the ‘elecs’ folder. This file can be loaded and the saved points can be plotted on the 3D surface mesh in MATLAB. This plot can be seen in the right panel. If the coordinates appear buried in the Mesh due to post operative brain shift, additional steps can be taken to project the electrode to the surface. 

**(B)** Example of identification of an electrode that is part of a subtemporal strip. The strip can be seen in the rCT.nii intensity projection map in the lower right panel. The coordinate is recorded from the center of the electrode artifact, seen in sagittal, coronal, and axial views. This coordinate can then be visualized on the 3D surface mesh in MATLAB, seen in the right panel.

### Note:
You need to name the hd_grid with <name>_corners.mat, which will then be able to be interpreted by patient.interp_grid


In [28]:
#This will open a GUI where you can click to mark the electrodes on the registered CT 
patient.mark_electrodes()

Launching electrode picker


#### 2f) Interpolate electrode grids

If you marked any grid corners, you should interpolate then project the electrodes.

In [28]:
help(patient.interp_grid)
patient.interp_grid(grid_basename='g',
                    ncols=8, nrows=6
                   )

Help on method interp_grid in module img_pipe.img_pipe.img_pipe:

interp_grid(nrows=16, ncols=16, grid_basename='hd_grid') method of img_pipe.img_pipe.img_pipe.freeCoG instance
    Interpolates corners for an electrode grid
    given the four corners (in order, 1, 16, 241, 256), or for
    32 channel grid, 1, 8, 25, 32.
    
    Parameters
    ----------
    nrows : int
        Number of rows in the grid
    ncols : int
        Number of columns in the grid
    grid_basename : str
        The base name of the grid (e.g. 'hd_grid' if you have a corners file
        called hd_grid_corners.mat)



#### 2g) Project electrodes that are buried under the cortical surface

This can either work on interpolated grids or individual electrodes. For interpolated grids, you have the option of using the mean normal vector as the projection direction (see figure). For individual electrodes, you can specify the projection direction if you set **`use_mean_normal=False`**. You can also select whether to use the dural surface or pial surface as the surface to project to.

<img src='files/ElectrodeProjection.png' style="width: 650px;">

**A.**  The grid’s corner electrodes are manually located. We interpolate the locations of the rest of the grid electrodes using these corner coordinates, giving us the electrode grid shown in red. The green arrows are the four normal vectors calculated from the corners, and the black arrow is the mean of those normal vectors and will act as our projection direction. 

**B.** Projection of the interpolated grid (red) to the convex hull of the pial surface (blue) using the mean normal vector (black arrow). The final projected electrode grid is shown in blue.


In [6]:
# help(patient.project_electrodes)

ModuleNotFoundError: No module named 'mcubes'

In [46]:
prefixes = [
#     'labt',
#     'llat',
#     'lpbt',
    'rabt',
    'rlat',
    'rpbt'
]

for prefix in prefixes:
    patient.project_electrodes(elecfile_prefix=prefix, dilate=10000., 
                           convex_hull=True, grid=False)

Projection Params: 
	 Grid Name: rabt.mat 
	 Use Mean Normal: True 
	                Surface Type: dural 
	 Number of Smoothing Iterations (if using dural): 30
::: Loading Mesh data :::
/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/umf005/Meshes/rh_dural_trivert.mat
::: Projecting electrodes to mesh :::
rh
::: Done :::
Projection Params: 
	 Grid Name: rlat.mat 
	 Use Mean Normal: True 
	                Surface Type: dural 
	 Number of Smoothing Iterations (if using dural): 30
::: Loading Mesh data :::
/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/umf005/Meshes/rh_dural_trivert.mat
::: Projecting electrodes to mesh :::
rh
::: Done :::
Projection Params: 
	 Grid Name: rpbt.mat 
	 Use Mean Normal: True 
	                Surface Type: dural 
	 Number of Smoothing Iterations (if using dural): 30
::: Loading Mesh data :::
/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/umf005/Meshes/rh_dura

In [10]:
patient.project_electrodes(elecfile_prefix='g', dilate=1000., convex_hull=True, grid=False)

Projection Params: 
	 Grid Name: g.mat 
	 Use Mean Normal: True 
	                Surface Type: dural 
	 Number of Smoothing Iterations (if using dural): 30
::: Loading Mesh data :::
/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/umf003/Meshes/lh_dural_trivert.mat
::: Projecting electrodes to mesh :::
lh
::: Done :::


#### Creating the **`*_elecs_all.mat`** file

Now we will create our **`'*_elecs_all.mat'`** montage file. This file is a `.mat` file with two structs: **`anatomy`**, which contains information about the montage labels, device origin, and anatomical labels; and **`elecmatrix`**, which contains the electrode coordinate matrix.

**`anatomy`** is structured as a __num_electrodes x 4__ matrix. The first column is the short name abbreviation of the electrode, the second column is the long name, the third column is the electrode type (i.e. depth, strip, or grid).

**`make_elecs_all()`** is an interactive function that creates the **`'*_elecs_all.mat'`** file.

In [47]:
# (short_name, long_name, elec_type, filename)
# umf001
# elecs_meta_input = [
#     ('btm', 'basal temporal medial', 'strip', 'btm.mat'),
#     ('btp', 'basal temporal pole', 'strip', 'btp.mat'),
#     ('c', 'grid64', 'grid', 'c.mat'),
# ]

# # umf 003
# elecs_meta_input = [
#     ('bo', 'basal occipital', 'strip', 'bo.mat'),
#     ('ph', 'hippocampus', 'depth', 'ph.mat'),
#     ('ah', 'anterior hippocampus', 'depth', 'ah.mat'),
#     ('g', 'grid48', 'grid', 'g.mat'),
# ]

# umf004
# elecs_meta_input = [
#     ('ltp', 'anterior temporal pole', 'depth', 'ltp.mat'),
#     ('la', 'right amygdala', 'depth', 'la.mat'),
#     ('lofs', 'orbitofrontal', 'depth', 'lofs.mat'),
#     ('lph', 'parahippocampal gyrus', 'depth', 'lph.mat'),
#     ('lh', 'hippocampus', 'depth', 'lh.mat'),
#     ('lmcsm', 'supplementary motor area', 'depth', 'lmcsm.mat'),
#     ('ra', 'right amygdala', 'depth', 'ra.mat'),
# ]

# umf005
elecs_meta_input = [
    ('lpbt', 'posterior basal temporal', 'strip', 'lpbt.mat'),
    ('labt', 'anterior basal temporal', 'strip', 'labt.mat'),
    ('llat', 'lateral temporal', 'strip', 'llat.mat'),
    ('rpbt', 'posterior basal temporal', 'strip', 'rpbt.mat'),
    ('rabt', 'anterior basal temporal', 'strip', 'rabt.mat'),
    ('rlat', 'lateral temporal', 'strip', 'rlat.mat'),
]



outfile = 'umf005_elecs_all'

print(outfile)

umf005_elecs_all


In [48]:
#creates the TDT_elecs_all.mat file 
patient.make_elecs_all(elecs_meta_input, outfile)

In [49]:
print(scipy.io.loadmat(os.path.join(patient.elecs_dir, f'{outfile}.mat'))['eleclabels'][0:5,:])
print(scipy.io.loadmat(os.path.join(patient.elecs_dir, f'{outfile}.mat'))['elecmatrix'].shape)

[[array(['lpbt1'], dtype='<U5')
  array(['posterior basal temporal1'], dtype='<U25')
  array(['strip'], dtype='<U5')]
 [array(['lpbt2'], dtype='<U5')
  array(['posterior basal temporal2'], dtype='<U25')
  array(['strip'], dtype='<U5')]
 [array(['lpbt3'], dtype='<U5')
  array(['posterior basal temporal3'], dtype='<U25')
  array(['strip'], dtype='<U5')]
 [array(['lpbt4'], dtype='<U5')
  array(['posterior basal temporal4'], dtype='<U25')
  array(['strip'], dtype='<U5')]
 [array(['labt1'], dtype='<U5')
  array(['anterior basal temporal1'], dtype='<U24')
  array(['strip'], dtype='<U5')]]
(28, 3)


In [50]:
#labels the electrodes and saves them to the 'anatomy' struct of the *_elecs_all.mat file 'desikan-killiany'
patient.label_elecs(elecfile_prefix=outfile, atlas_surf='destrieux', atlas_depth='destrieux')

/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/
Creating labels from the freesurfer annotation file for use in automated electrode labeling
Loading electrode matrix
Loading label rostralmiddlefrontal
Loading label bankssts
Loading label lateralorbitofrontal
Loading label inferiortemporal
Loading label posteriorcingulate
Loading label parsopercularis
Loading label parsorbitalis
Loading label middletemporal
Loading label inferiorparietal
Loading label supramarginal
Loading label postcentral
Loading label lateraloccipital
Loading label pericalcarine
Loading label rostralanteriorcingulate
Loading label fusiform
Loading label insula
Loading label lingual
Loading label transversetemporal
Loading label caudalanteriorcingulate
Loading label parahippocampal
Loading label caudalmiddlefrontal
Loading label superiorfrontal
Loading label cuneus
Loading label isthmuscingulate
Loading label precentral
Loading label medialorbitofrontal
Loading label temporalpole
Loadi

array([['lpbt1', 'posterior basal temporal1', 'strip', 'Unknown'],
       ['lpbt2', 'posterior basal temporal2', 'strip', 'Unknown'],
       ['lpbt3', 'posterior basal temporal3', 'strip', 'Unknown'],
       ['lpbt4', 'posterior basal temporal4', 'strip', 'Unknown'],
       ['labt1', 'anterior basal temporal1', 'strip', 'Unknown'],
       ['labt2', 'anterior basal temporal2', 'strip', 'Unknown'],
       ['labt3', 'anterior basal temporal3', 'strip',
        'medialorbitofrontal'],
       ['labt4', 'anterior basal temporal4', 'strip',
        'medialorbitofrontal'],
       ['llat1', 'lateral temporal1', 'strip', 'medialorbitofrontal'],
       ['llat2', 'lateral temporal2', 'strip', 'medialorbitofrontal'],
       ['llat3', 'lateral temporal3', 'strip', 'medialorbitofrontal'],
       ['llat4', 'lateral temporal4', 'strip', 'medialorbitofrontal'],
       ['llat5', 'lateral temporal5', 'strip', 'medialorbitofrontal'],
       ['llat6', 'lateral temporal6', 'strip', 'Unknown'],
       ['rpbt1

In [54]:
patient.warp_all(elecfile_prefix=outfile)

Using cvs_avg35_inMNI152 as the template for warps
::: Computing Non-linear warping from patient native T1 to fs CVS MNI152 :::
cvsWarp COMPUTED
:::: Computing RAS2Vox and saving umf003_elecs_all txt file ::::
:::: Applying Non-linear warping ::::
:::: Computing Vox2RAS and saving umf003_elecs_all_warped mat file ::::


OSError: /Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/umf003/elecs/umf003_elecs_all_nearest_warped.txt not found.

### Final File Structure

SUBJECTS_DIR/
    * Subj_name/
        * CT/	
        * Meshes/	
        * acpc/	
        * ascii/	
        * cvs/
        * dicom/	
        * elecs/
            -individual_elecs/
        * label/	
        * mri/	
        * scripts/	
        * surf/	

See plotting demo.